Alignement des entités nommées avec la base de données Wikidata et récupération des informations utiles à l'étude:
- Type de lieu
- Données de géolocalisation (latitude et longitude)
- Description du lieu

Outils: qwikidata, wptools

In [ ]:
# @title Installation des biliothèques et connexion au Google Drive

"""Marina Giardinetti
marina.giardinetti@gmail.com
Google Colab
2024"""

!pip install qwikidata
import qwikidata
!pip install wptools
import wptools
import os
import pandas as pd
from collections import Counter
from lxml.etree import XMLSyntaxError
from json import JSONDecodeError
import re
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)

from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# @title Création de nouveaux fichiers .xlsx contenant les informations récupérées
def get_infobox(donnees, itemid):
  type_lieu, latitude, longitude, description = ([] for i in range(4))
  page = wptools.page(wikibase=itemid, lang="fr")
  page.get_wikidata()
  try:
    page.get_parse()
    infobox = page.data['infobox']
    tout=page.data['wikidata']
    infobox=page.data['infobox']
    nom = infobox.get("nom")
    type_lieu=infobox.get("type")
    if type_lieu is None:
        type_lieu=page.data['what']
    latitude = infobox.get("latitude")
    longitude = infobox.get("longitude")
    description = page.data["description"]
    if latitude is None:
      pass
    else:
      donnees[nom]= {"Type de lieu" : type_lieu, "Latitude" : latitude, "Longitude": longitude, 'Description': description, 'URL': uri}
  except (AttributeError, LookupError, XMLSyntaxError):pass



def get_ID(title):
    title = title.lower().title()
    title = re.sub('(la )|(les )|(le )', '', title)
    sparql_query = ''.join(['SELECT * WHERE {?sujet rdfs:label \"', title, '\"@fr.?sujet ?propriete ?valeur.}'])
    try:
      res = return_sparql_query_results(sparql_query)
      for key, value in res.items():
        if key == 'results':
          for key2, value2 in value.items():
            if len(value2)==0:
              pass
            else:
              no1 = value2[0]
              sujet =no1.get("sujet")
              global uri
              uri = sujet.get("value")
              ID = uri.split('/')[4]
              get_infobox(donnees, ID)
    except JSONDecodeError:pass

donnees={}
working_dir = '/content/drive/My Drive/NER_BNF/'#@param {type:"string"}
xls_file = '/content/drive/My Drive/NER_BNF/bert_lieux_sample.xlsx' #@param {type:"string"}
colonne_entites = 'word'#@param {type:"string"}
with open(xls_file, "rb") as myfile:
  df = pd.read_excel(myfile)
  lieux = df["word"].values.tolist()

for lieu in lieux:
  get_ID(title=lieu)

nom_fichier = 'lieux_bert_complets.xlsx' #@param {type:"string"}
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel("".join([working_dir, nom_fichier]))

In [ ]:
# @title Études statistiques

# Statistiques / types de lieux
#@markdown Ouvrir un fichier .XLSX pour effectuer les calculs statistiques

fichier_xlsx = '/content/drive/My Drive/NER_BNF/lieux_bert_complets.xlsx' #@param {type:"string"}
with open(fichier_xlsx, 'rb') as myFile:
  df3=pd.read_excel(myFile)
  liste_des_types = df3["Type de lieu"].values.tolist()

def proportion_types(liste):
  c = Counter(liste)
  print("Voici le nombre de lieux, triés par chaque type:", c) ## Affiche le nombre de lieux de chaque type
  prop = [(i, c[i] / len(liste) * 100.0) for i in c]
  print("Voici la proportion de chaque type de lieux:", prop) ## Affiche la proportion de chaque type de lieux
  df_genre = pd.DataFrame.from_dict(c, orient='index').reset_index()
  df_genre.rename(columns={'index': 'Type de lieu'}, inplace=True)
  display(df_genre)

proportion_types(liste_des_types)